In [1]:
import time
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys

sys.path.insert(0, "../")

from app.mlcode.predict_price_movements import CoinPricePredictor
from app.mlcode.determine_trading_state import DetermineTradingState
from app.mlcode.utils import read_in_data, read_in_yaml, running_on_aws, setup_logging, update_yaml_config

%pylab inline

import logging

from tqdm.notebook import tqdm

tqdm.pandas()

logger = logging.getLogger(__name__)

logger.setLevel(logging.CRITICAL)

import warnings

warnings.simplefilter("ignore")

Importing plotly failed. Interactive plots will not work.


Populating the interactive namespace from numpy and matplotlib


/Users/jonathanhilgart/Library/Caches/pypoetry/virtualenvs/go-trader-aTWd7ZB6-py3.8/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
logging.disable(sys.maxsize)

## Setup data
- All of these configs are stored in the notebooks folder

In [25]:
def read_in_config_files(base_path: str = "configs/", coin: str = "btc"):

    coin_to_predict = coin + "_"

    constants = read_in_yaml(base_path + "constants.yml", running_on_aws())

    trading_state_filename = base_path + coin_to_predict + "trading_state_config.yml"
    trading_constants = read_in_yaml(trading_state_filename, running_on_aws())

    won_lost_amount_filename = base_path + coin_to_predict + "won_and_lost_config.yml"
    won_and_lost_amount_constants = read_in_yaml(won_lost_amount_filename, running_on_aws())

    actions_to_take_filename = base_path + coin_to_predict + "actions_to_take.yml"
    actions_to_take_constants = read_in_yaml(actions_to_take_filename, running_on_aws())

    ml_constants = read_in_yaml(base_path + constants["ml_config_filename"].split("/")[1], running_on_aws())

    return coin, constants, trading_constants, won_and_lost_amount_constants, actions_to_take_constants, ml_constants

In [26]:
(
    coin,
    constants,
    trading_constants,
    won_and_lost_amount_constants,
    actions_to_take_constants,
    ml_constants,
) = read_in_config_files(coin="btc")

In [4]:
trading_constants  # ensure all zeros
assert trading_constants["buy_entry_price"] == 0
assert trading_constants["position_entry_date"] == None
trading_constants

{'buy_entry_price': 0.0,
 'buy_has_crossed_mean': 0.0,
 'mode': 'no_position',
 'position_entry_date': None,
 'short_entry_price': 0.0,
 'short_has_crossed_mean': 0.0,
 'stop_loss_pct': 0.1,
 'stop_loss_price': 0.0}

In [16]:
won_and_lost_amount_constants

{'dollar_amount_buy_lost': 0.0,
 'dollar_amount_buy_won': 0.0,
 'dollar_amount_short_lost': 0.0,
 'dollar_amount_short_won': 0.0,
 'n_buy_lost': 0.0,
 'n_buy_won': 0.0,
 'n_short_lost': 0.0,
 'n_short_won': 0.0,
 'n_total_days_in_trades': 0.0}

In [17]:
bitcoin_df = read_in_data("../tmp/" + constants["bitcoin_csv_filename"].split("/")[1], running_on_aws())
etherum_df = read_in_data("../tmp/" + constants["etherum_csv_filename"].split("/")[1], running_on_aws())
sol_df = read_in_data("../tmp/" + constants["sol_csv_filename"].split("/")[1], running_on_aws())
matic_df = read_in_data("../tmp/" + constants["matic_csv_filename"].split("/")[1], running_on_aws())
link_df = read_in_data("../tmp/" + constants["link_csv_filename"].split("/")[1], running_on_aws())

In [18]:
ml_constants

{'prediction_params': {'bollinger_window': 5,
  'no_of_std': 1.25,
  'lookback_window': [15, 30, 45],
  'prediction_n_days': 7,
  'model_name': ['TCN', 'NBEATS'],
  'work_dir': '/mnt/shared-storage'},
 'hyperparameters_tcn': {'dropout': 0.1,
  'random_state': 0,
  'dilation_base': 2,
  'kernel_size': 14,
  'num_filters': 7,
  'num_layers': 7,
  'weight_norm': True,
  'epochs': 55},
 'hyperparameters_nbeats': {'random_state': 0,
  'num_blocks': 5,
  'layer_widths': 256,
  'epochs': 10}}

 - BTC

## SEtup the two main classes
- predict prices
- update trading state (bollinger bands)

In [19]:
# ml_constants['hyperparameters_tcn']['epochs']=1
# ml_constants['hyperparameters_nbeats']['epochs']=1

In [20]:
ml_constants

{'prediction_params': {'bollinger_window': 5,
  'no_of_std': 1.25,
  'lookback_window': [15, 30, 45],
  'prediction_n_days': 7,
  'model_name': ['TCN', 'NBEATS'],
  'work_dir': '/mnt/shared-storage'},
 'hyperparameters_tcn': {'dropout': 0.1,
  'random_state': 0,
  'dilation_base': 2,
  'kernel_size': 14,
  'num_filters': 7,
  'num_layers': 7,
  'weight_norm': True,
  'epochs': 55},
 'hyperparameters_nbeats': {'random_state': 0,
  'num_blocks': 5,
  'layer_widths': 256,
  'epochs': 10}}

### Write a loop going through each day in our main df
- Filter so that the largest lookback window is satisfied
- run price prediction and determine trading state
- go on to next date
- ..etc

In [21]:
max_lookback = np.max(ml_constants["prediction_params"]["lookback_window"])  # days

In [22]:
max_lookback

45

In [23]:
trading_state_class.trading_state_constants

{'buy_entry_price': 0.0,
 'buy_has_crossed_mean': 0.0,
 'mode': 'no_position',
 'position_entry_date': None,
 'short_entry_price': 0.0,
 'short_has_crossed_mean': 0.0,
 'stop_loss_pct': 0.1,
 'stop_loss_price': 0.0}

In [ ]:
for day_idx, day in enumerate(bitcoin_df.index):

    if (day_idx) > max_lookback * 2:
        print(f" Pct complete = {day_idx/len(bitcoin_df):.2f}")
        new_bitcoin_df = bitcoin_df[bitcoin_df.index <= day]
        new_etherum_df = etherum_df[etherum_df.index <= day]
        # read in the new config files in case the yhave been updated
        (
            coin,
            constants,
            trading_constants,
            won_and_lost_amount_constants,
            actions_to_take_constants,
            ml_constants,
        ) = read_in_config_files(coin="btc")

        predictor = CoinPricePredictor(
            coin, constants, ml_constants, new_bitcoin_df, additional_dfs=[new_etherum_df], verbose=False  # spy_df
        )

        price_prediction = predictor.predict()

        trading_state_class = DetermineTradingState(
            coin,
            price_prediction,
            constants,
            trading_constants,
            predictor.df,
            won_and_lost_amount_constants,
            actions_to_take_constants,
            running_on_aws(),
            is_training=True,
        )
        # TODO: read in yaml files again
        sys.stdout.flush()
        trading_state_class.calculate_positions()
        logger.info("---- Finished determinig trading strategy --- ")
        trading_state_class.update_state()
        # this works
        if trading_state_class.actions_to_take_constants["action_to_take"] != "none_to_none":
            print(trading_state_class.trading_state_constants, "trading_state_class.trading_state_constants")
            print(trading_state_class.actions_to_take_constants, "trading_state_class.actions_to_take_constants")

        update_yaml_config(trading_state_filename, trading_state_class.trading_state_constants, running_on_aws())
        logger.info("---- Updated trading state config --- ")
        update_yaml_config(won_lost_amount_filename, trading_state_class.won_and_lose_amount_dict, running_on_aws())
        logger.info("---- Updated win/lost state config --- ")
        update_yaml_config(actions_to_take_filename, trading_state_class.actions_to_take_constants, running_on_aws())
        logger.info("---- Updated actions to take state config --- ")

 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.05
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.06
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete = 0.07
 Pct complete

In [41]:
new_eth_df

,open,high,low,close,volume
date,,,,,
2017-01-01,7.98,8.47,7.98,8.17,14731700.0
2017-01-02,8.17,8.44,8.05,8.38,14579600.0
2017-01-03,8.37,10.00,8.32,9.73,33625200.0
2017-01-04,9.71,11.28,9.56,11.25,41051200.0
2017-01-05,11.29,11.89,9.40,10.25,41557400.0
2017-01-06,10.29,10.63,9.63,10.25,29471800.0
2017-01-07,10.24,10.28,9.59,9.87,23153600.0
2017-01-08,9.87,10.39,9.83,10.29,16676600.0
2017-01-09,10.31,10.78,10.14,10.33,25718300.0


In [42]:
new_bitcoin_df

,open,high,low,close,volume
date,,,,,
2017-01-01,963.66,1003.08,958.70,998.33,147775008.0
2017-01-02,998.62,1031.39,996.70,1021.75,222184992.0
2017-01-03,1021.60,1044.08,1021.60,1043.84,185168000.0
2017-01-04,1044.40,1159.42,1044.40,1154.73,344945984.0
2017-01-05,1156.73,1191.10,910.42,1013.38,510199008.0
...,...,...,...,...,...
2017-03-29,1046.08,1055.13,1015.88,1039.97,298457984.0
2017-03-30,1042.21,1049.29,1020.04,1026.43,352968992.0
2017-03-31,1026.64,1074.92,1026.64,1071.79,447287008.0


2022-04-01 15:28:15,032: : predict  Slicing dataframes
2022-04-01 15:28:15,034: : _slice_df  Slice date , earliest day of data for main df or years filter = 2017-01-01 00:00:00
2022-04-01 15:28:15,036: : predict  Building Bollinger Bands
2022-04-01 15:28:15,042: : _build_technical_indicators  ---- Adding Bollinger Bands ----
2022-04-01 15:28:15,043: : _build_technical_indicators                 open     high      low    close       volume  Rolling Mean  \
date                                                                        
2017-03-29  1046.08  1055.13  1015.88  1039.97  298457984.0      1014.478   
2017-03-30  1042.21  1049.29  1020.04  1026.43  352968992.0      1025.208   
2017-03-31  1026.64  1074.92  1026.64  1071.79  447287008.0      1046.222   
2017-04-01  1071.71  1091.72  1061.09  1080.50  289633984.0      1053.168   
2017-04-02  1080.61  1107.59  1075.45  1102.17  514187008.0      1064.172   

            Bollinger High  Bollinger Low  
date                             

  0%|          | 0/1 [00:00<?, ?it/s]

[2022-04-01 15:28:15,791] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-01 15:28:15,791] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-04-01 15:28:15,791: : _init_model  Time series values are 64-bits; casting model to float64.
[2022-04-01 15:28:15,856] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-04-01 15:28:15,856] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.


  0%|          | 0/1 [00:00<?, ?it/s]

2022-04-01 15:28:15,856: : _init_model  Time series values are 64-bits; casting model to float64.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

2022-04-01 15:28:31,632: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x127f80a30>, <darts.models.forecasting.tcn_model.TCNModel object at 0x128d35670>]_tcn
2022-04-01 15:28:31,634: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x128cf7d60>, <darts.models.forecasting.tcn_model.TCNModel object at 0x128cf7eb0>]_nbeats
2022-04-01 15:28:31,635: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x128cf7d60>, <darts.models.forecasting.tcn_model.TCNModel object at 0x128cf7eb0>]_tcn
2022-04-01 15:28:31,635: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x12678b070>, <darts.models.forecasting.tcn_model.TCNModel object at 0x128d0ed60>]_nbeats
2022-04-01 15:28:31,636: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x12678b070>, <darts.models.forecasting.tcn_model.TCNModel object at 0x128d0ed60>]_tcn

2022-04-01 15:28:31,726: : _make_prediction   Model = nbeats_btc_lookback_15 Lookback = 7 Prediction = 1280.9202244049013
2022-04-01 15:28:31,737: : _make_prediction   Model = tcn_btc_lookback_15 Lookback = 7 Prediction = -17.899056131425368
2022-04-01 15:28:31,815: : _make_prediction   Model = nbeats_btc_lookback_30 Lookback = 7 Prediction = 2274.703129051977
2022-04-01 15:28:31,826: : _make_prediction   Model = tcn_btc_lookback_30 Lookback = 7 Prediction = 45.754998042634355
2022-04-01 15:28:31,936: : _make_prediction   Model = nbeats_btc_lookback_45 Lookback = 7 Prediction = 2881.4894364841352
2022-04-01 15:28:31,951: : _make_prediction   Model = tcn_btc_lookback_45 Lookback = 7 Prediction = 2.599315684614381
2022-04-01 15:28:31,952: : predict  prediction = 1077.9280079228063


2022-04-01 15:35:49,645: : __init__  Setting the class var self.buy_entry_price = 45520.0
2022-04-01 15:35:49,649: : __init__  Setting the class var self.buy_has_crossed_mean = 0.0
2022-04-01 15:35:49,650: : __init__  Setting the class var self.mode = buy
2022-04-01 15:35:49,651: : __init__  Setting the class var self.position_entry_date = 2022-03-31
2022-04-01 15:35:49,653: : __init__  Setting the class var self.short_entry_price = 0.0
2022-04-01 15:35:49,655: : __init__  Setting the class var self.short_has_crossed_mean = 0.0
2022-04-01 15:35:49,656: : __init__  Setting the class var self.stop_loss_pct = 0.1
2022-04-01 15:35:49,660: : __init__  Setting the class var self.stop_loss_price = 40968.0
2022-04-01 15:35:49,663: : __init__  Setting the class var self.dollar_amount_buy_lost = 0.0
2022-04-01 15:35:49,665: : __init__  Setting the class var self.dollar_amount_buy_won = 2813.0
2022-04-01 15:35:49,672: : __init__  Setting the class var self.dollar_amount_short_lost = 0.0
2022-04-0

### Update configs

2022-04-01 09:51:07,611: : calculate_positions  current row =                open     high      low    close        volume  Rolling Mean  \
date                                                                         
2022-03-31  47085.0  47680.0  45133.0  45520.0  7.928625e+07       46812.6   

            Bollinger High  Bollinger Low    stc      stoch        rsi  \
date                                                                     
2022-03-31    47756.439234   45868.760766  100.0  66.480447  51.345275   

                   macd  macd_signal  
date                                  
2022-03-31  1700.536619  1255.298319  
2022-04-01 09:51:07,622: : calculate_positions  prev_row =                open     high      low    close        volume  Rolling Mean  \
date                                                                         
2022-03-30  47455.0  47716.0  46329.0  47085.0  5.690887e+07       46614.8   

            Bollinger High  Bollinger Low    stc      stoch        rs